# Presentación: Juego "Hundir la Flota"

Este notebook tiene como propósito explicar de manera exhaustiva el código del juego **Hundir la Flota**. 
El proyecto se divide en dos archivos:

- **`funciones.py`**: Contiene la clase principal `Tablero` con toda la lógica del juego.
- **`main.py`**: Es el punto de entrada del programa que ejecuta el flujo del juego.

A lo largo de este notebook, describiremos cada función y método paso a paso, con fragmentos de código y explicaciones.


## Estructura del Proyecto

El proyecto consta de los siguientes archivos:

1. **`funciones.py`**: Este archivo define la clase `Tablero`, que es la base del juego. 
   Incluye los métodos para gestionar tableros, barcos, disparos y la interacción del juego.
   
2. **`main.py`**: Es el programa principal que inicializa una instancia de la clase `Tablero` 
   y gestiona el flujo del juego interactivo entre el usuario y la máquina.


## Clase `Tablero`

La clase `Tablero` es el componente principal que define las reglas y operaciones del juego. 
En esta sección, describiremos los atributos y todos los métodos implementados en esta clase.


### Atributos de la Clase

La clase `Tablero` inicializa los siguientes atributos:

- **`dimensiones`**: Dimensiones del tablero (por defecto, 10x10).
- **`id_user`** y **`id_machine`**: Identificadores únicos para el jugador y la máquina.
- **`tablero_user`** y **`tablero_machine`**: Tableros representados como matrices de caracteres.
- **`disparos_realizados_user`** y **`disparos_realizados_machine`**: Conjuntos que almacenan las coordenadas de los disparos realizados.
- **`barcos_user`** y **`barcos_machine`**: Diccionarios que contienen las posiciones de los barcos para cada jugador.

Código correspondiente:


In [1]:
class Tablero:
    def __init__(self, dimensiones=(10, 10), id_user=1, id_machine=0):
        self.dimensiones = dimensiones
        self.id_user = id_user
        self.id_machine = id_machine
        self.tablero_user = np.full(dimensiones, "~")  # Tablero del usuario
        self.tablero_machine = np.full(dimensiones, "~")  # Tablero de la máquina
        self.disparos_realizados_user = set()  # Disparos realizados por el usuario
        self.disparos_realizados_machine = set()  # Disparos realizados por la máquina
        self.barcos_user = {}  # Diccionario con los barcos del usuario
        self.barcos_machine = {}  # Diccionario con los barcos de la máquina


### Métodos para Colocación de Barcos

Los siguientes métodos se encargan de generar posiciones aleatorias para los barcos y ubicarlos en los tableros:

1. **`crear_barcos_aleatorios`**: Genera posiciones aleatorias para los barcos según las esloras y cantidades especificadas.
2. **`intentar_colocar_barco`**: Intenta colocar un barco en el tablero de manera válida.
3. **`colocar_barcos_en_tablero`**: Coloca físicamente las posiciones generadas en el tablero.


In [ ]:
def crear_barcos_aleatorios(self, tablero, esloras=[1, 2, 3, 4], cantidades=[4, 3, 2, 1], max_intentos=100):
    barcos_dict = {}
    for eslora, cantidad in zip(esloras, cantidades):
        for n in range(cantidad):
            posiciones = self.intentar_colocar_barco(tablero, eslora, max_intentos)
            if posiciones:
                barcos_dict[f"barco{eslora}_{n+1}"] = posiciones
    return barcos_dict

def intentar_colocar_barco(self, tablero, eslora, max_intentos=100):
    for _ in range(max_intentos):
        fila_inicial, columna_inicial, orientacion = self.obtener_posicion_aleatoria(eslora)
        posiciones = [(fila_inicial, columna_inicial)]
        if self.posicion_valida(tablero, posiciones, orientacion, eslora):
            for _ in range(1, eslora):
                fila_inicial, columna_inicial = self.siguiente_posicion(posiciones[-1][0], posiciones[-1][1], orientacion)
                posiciones.append((fila_inicial, columna_inicial))
            self.colocar_barcos_en_tablero({f"barco{eslora}": posiciones}, tablero)
            return posiciones
    return None

def colocar_barcos_en_tablero(self, barcos_dict, tablero):
    for posiciones in barcos_dict.values():
        for fila, columna in posiciones:
            tablero[fila, columna] = "O"


### Método `obtener_posicion_aleatoria`

Este método genera una posición inicial aleatoria en el tablero para colocar un barco. También asigna una orientación (Norte, Sur, Este, Oeste) de manera aleatoria.

#### Pasos:
1. **Generar coordenadas aleatorias**:
   - Se utiliza `random.randint` para obtener una fila y una columna dentro de las dimensiones del tablero.
2. **Elegir orientación**:
   - Se selecciona aleatoriamente una orientación usando `random.choice`.
3. **Retornar los valores**:
   - Devuelve las coordenadas iniciales y la orientación.

#### Código:


In [ ]:
def obtener_posicion_aleatoria(self, eslora):
    fila_inicial = random.randint(0, self.dimensiones[0] - 1)
    columna_inicial = random.randint(0, self.dimensiones[1] - 1)
    orientacion = random.choice(["N", "S", "O", "E"])
    return fila_inicial, columna_inicial, orientacion


### Método `posicion_valida`

Verifica si un barco puede colocarse en una posición inicial y orientación específica sin salirse de los límites del tablero ni superponerse con otro barco.

#### Pasos:
1. **Iterar sobre la longitud del barco**:
   - Se calculan las posiciones sucesivas basadas en la orientación.
2. **Validar posición**:
   - Comprueba que las posiciones estén dentro de los límites del tablero.
   - Verifica que las casillas estén vacías (`~`).
3. **Retornar resultado**:
   - Si todas las posiciones son válidas, devuelve `True`. En caso contrario, `False`.

#### Código:


In [ ]:
def posicion_valida(self, tablero, posiciones, orientacion, eslora):
    for _ in range(1, eslora):
        fila_inicial, columna_inicial = self.siguiente_posicion(posiciones[-1][0], posiciones[-1][1], orientacion)
        if not (0 <= fila_inicial < self.dimensiones[0] and 0 <= columna_inicial < self.dimensiones[1]):
            return False
        if tablero[fila_inicial, columna_inicial] != "~":
            return False
        posiciones.append((fila_inicial, columna_inicial))
    return True


### Método `siguiente_posicion`

Calcula la siguiente posición en el tablero basándose en la orientación y una posición inicial.

#### Pasos:
1. **Mapeo de direcciones**:
   - Define un diccionario `direcciones` con las transformaciones para cada orientación.
2. **Calcular posición siguiente**:
   - Aplica la transformación correspondiente a la orientación para obtener la nueva fila y columna.
3. **Retornar posición**:
   - Devuelve las coordenadas calculadas.

#### Código:


In [ ]:
def siguiente_posicion(self, fila, columna, orientacion):
    direcciones = {
        "N": (-1, 0),
        "S": (1, 0),
        "E": (0, 1),
        "O": (0, -1),
    }
    return fila + direcciones[orientacion][0], columna + direcciones[orientacion][1]


### Métodos `disparar_user` y `disparar_machine`

Ambos métodos realizan un disparo en el tablero del oponente y actualizan el tablero según el resultado.

#### Pasos comunes:
1. **Verificar disparo repetido**:
   - Comprueban si las coordenadas ya han sido atacadas.
2. **Actualizar tablero**:
   - Si hay un barco (`O`), se marca como impacto (`X`).
   - Si no, se marca como fallo (`*`).

#### Código de `disparar_user`:


In [ ]:
def disparar_user(self, fila, columna, tablero_oponente):
    if (fila, columna) in self.disparos_realizados_user:
        print("Ya habías intentado aquí")
    self.disparos_realizados_user.add((fila, columna))
    if tablero_oponente[fila, columna] == "O":
        tablero_oponente[fila, columna] = "X"
        print("Impacto")
    else:
        tablero_oponente[fila, columna] = "*"
        print("Fallaste")


In [ ]:
def disparar_machine(self, fila, columna, tablero_oponente):
    if (fila, columna) in self.disparos_realizados_machine:
        print("Ya habías intentado aquí")
    self.disparos_realizados_machine.add((fila, columna))
    if tablero_oponente[fila, columna] == "O":
        tablero_oponente[fila, columna] = "X"
        print("Impacto")
    else:
        tablero_oponente[fila, columna] = "*"
        print("Fallaste")


### Método `tocado_hundido`

Determina si un barco ha sido tocado o hundido después de un impacto.

#### Pasos:
1. **Recorrer barcos**:
   - Busca en el diccionario de barcos si el impacto pertenece a alguna de las coordenadas.
2. **Contar impactos**:
   - Compara la cantidad de impactos en el barco con su longitud.
3. **Determinar estado**:
   - Si todas las coordenadas del barco están impactadas, está hundido.
   - Si no, el barco solo está tocado.

#### Código:


In [ ]:
def tocado_hundido(self, impacto, tablero, barcos_dict):
    impacto = tuple(impacto)
    for barco, coordenadas in barcos_dict.items():
        longitud_barco = len(coordenadas)
        impactos = sum(1 for coordenada in coordenadas if tablero[coordenada] == "X")
        if impacto in coordenadas:
            if impactos == longitud_barco:
                print("Barco Hundido")
            else:
                print("Barco Tocado")


### Método `imprimir_tablero`

Imprime el tablero en consola, ocultando los barcos si se especifica.

#### Pasos:
1. **Generar encabezado**:
   - Crea las etiquetas de las columnas (A, B, C...).
2. **Recorrer filas**:
   - Imprime cada fila, ocultando los barcos (`O`) si es necesario.
3. **Mostrar tablero**:
   - Formatea y muestra el tablero.

#### Código:


In [ ]:
def imprimir_tablero(self, tablero, ocultar_barcos=False):
    columnas = list(string.ascii_uppercase[:self.dimensiones[1]])
    print("  " + " ".join(columnas))
    for i, fila in enumerate(tablero):
        fila_impresa = [
            "~" if (casilla == "O" and ocultar_barcos) else casilla for casilla in fila
        ]
        print(f"{i + 1:2} " + " ".join(fila_impresa))  # La numeración empieza desde 1


### Método `obtener_coordenadas_usuario`

Solicita al usuario que introduzca las coordenadas de un disparo y valida la entrada.

#### Pasos:
1. **Pedir entrada al usuario**:
   - Solicita un disparo en formato "A1", "B3", etc.
2. **Validar entrada**:
   - Verifica que la columna y fila sean válidas.
3. **Retornar coordenadas**:
   - Convierte la entrada en índices de matriz y los devuelve.

#### Código:


In [ ]:
def obtener_coordenadas_usuario(self):
    while True:
        disparo = input("Ingresa las coordenadas del disparo (Ejemplos: A1, B3): ").upper()
        if len(disparo) >= 2 and disparo[0] in string.ascii_uppercase[:self.dimensiones[1]] and disparo[1:].isdigit():
            columna = string.ascii_uppercase.index(disparo[0])
            fila = int(disparo[1:]) - 1
            if 0 <= fila < self.dimensiones[0] and 0 <= columna < self.dimensiones[1]:
                return fila, columna
        print("Coordenada inválida. Intenta de nuevo.")


### Método `iniciar_juego`

Este es el método principal que gestiona la interacción entre el usuario y la máquina durante el juego. Sus pasos principales son:

1. **Bienvenida e inicialización**: Muestra un mensaje de bienvenida y prepara los tableros de ambos jugadores.
2. **Colocación de barcos**:
   - Genera posiciones aleatorias para los barcos del usuario y de la máquina.
   - Muestra el tablero del usuario con los barcos visibles y el tablero de la máquina con los barcos ocultos.
3. **Gestión del turno**:
   - **Turno del usuario**:
     - Solicita las coordenadas para disparar.
     - Realiza el disparo y muestra el resultado (impacto o fallo).
   - **Turno de la máquina**:
     - Genera coordenadas aleatorias para disparar.
     - Realiza el disparo y muestra el resultado (impacto o fallo).
4. **Comprobación del estado del juego**: Al final de cada turno, verifica si todos los barcos de algún jugador han sido destruidos.
5. **Finalización del juego**: Declara al ganador o termina si el usuario decide salir.

Código correspondiente:


In [ ]:
def iniciar_juego(self):
    # Muestra un mensaje de bienvenida
    print("\n¡Bienvenido a Hundir la Flota!")
    print("A continuación, podrás disparar dentro de diferentes coordenadas.")
    print("Las coordenadas van de A1 a J10.")
    
    # Colocación de barcos
    self.barcos_user = self.crear_barcos_aleatorios(self.tablero_user)
    self.barcos_machine = self.crear_barcos_aleatorios(self.tablero_machine)
    
    print("\nTablero del usuario:")
    self.imprimir_tablero(self.tablero_user)
    
    print("\nTablero de la máquina (escondido):")
    self.imprimir_tablero(self.tablero_machine, ocultar_barcos=True)

    # Inicio del bucle del juego
    while not self.comprobar_fin_de_juego():
        print("\nTu turno")
        # Solicitar coordenadas al usuario
        coordenadas = self.obtener_coordenadas_usuario()
        if coordenadas is None:  # El usuario puede salir del juego
            print("Gracias por jugar.")
            break

        fila, columna = coordenadas
        # Realizar disparo del usuario
        resultado = self.realizar_disparo(fila, columna, self.tablero_machine, self.disparos_realizados_user, self.barcos_machine)
        print(f"Resultado: {resultado}")

        print("\nTurno de la máquina")
        # Generar disparo aleatorio para la máquina
        fila, columna = random.randint(0, self.dimensiones[0] - 1), random.randint(0, self.dimensiones[1] - 1)
        print(f"La máquina disparó en {string.ascii_uppercase[columna]}{fila+1}")
        resultado = self.realizar_disparo(fila, columna, self.tablero_user, self.disparos_realizados_machine, self.barcos_user)
        print(f"Resultado de la máquina: {resultado}")

        # Imprimir los tableros actualizados
        print("\nTablero del usuario:")
        self.imprimir_tablero(self.tablero_user)
        
        print("\nTablero de la máquina (escondido):")
        self.imprimir_tablero(self.tablero_machine, ocultar_barcos=True)


#### Detalles adicionales:

1. **Bienvenida e inicialización**:
   - Se utiliza `print` para mostrar mensajes iniciales al usuario.
   - Los barcos se colocan de forma aleatoria en los tableros mediante el método `crear_barcos_aleatorios`.

2. **Gestión del turno del usuario**:
   - Utiliza el método `obtener_coordenadas_usuario` para solicitar coordenadas válidas.
   - El disparo se realiza con `realizar_disparo`, que actualiza el tablero de la máquina y verifica si se ha hundido un barco.

3. **Turno de la máquina**:
   - Genera coordenadas aleatorias para disparar en el tablero del usuario.
   - Al igual que el usuario, el disparo se gestiona con `realizar_disparo`.

4. **Verificación del estado del juego**:
   - El método `comprobar_fin_de_juego` verifica si todos los barcos de un jugador han sido destruidos.

5. **Finalización**:
   - Si el usuario decide salir, se muestra un mensaje de despedida.
   - Si se hunden todos los barcos de un jugador, se declara al ganador.

Este método combina los componentes principales del juego, proporcionando una experiencia interactiva por turnos.


## Flujo del Programa (`main.py`)

El archivo `main.py` inicializa el juego y gestiona el flujo principal. A continuación se describe el código.


In [ ]:
from funciones import *
import random
import numpy as np

if __name__ == "__main__":
    juego = Tablero()
    juego.iniciar_juego()


## Conclusión y Posibles Mejoras

El juego *Hundir la Flota* está implementado utilizando principios básicos de programación en Python. 
Algunas posibles mejoras incluyen:

- Incorporar una interfaz gráfica de usuario para hacerlo más atractivo visualmente.
- Implementar un sistema de dificultad ajustable para la máquina.
- Que los barcos no solapen entre sí al ser colocados en el tablero y no haya barcos adyacentes.

Este modelo es una excelente base para aprender sobre programación orientada a objetos y diseño de juegos por turnos.
